## Transorming pitstops data and storing it in silver layer for future transformations

####  using run commands to access notebook to run codes to configuring the adlsgen2 and connect|

In [0]:
%run "../includes/adlsconfig"

In [0]:
%run "../includes/containersconfig"

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
pitstop_struct_schema = StructType([
                                    StructField("raceId", IntegerType(), False),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("stop", StringType(), True),
                                    StructField("lap", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("duration", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True)
])

In [0]:
pitstop_df =  spark.read.format('json').option('multiline', True).schema(pitstop_struct_schema)\
    .load(f"{raw_folder_path}/pit_stops.json")

In [0]:
pitstop_final_df = pitstop_df.withColumnsRenamed({"raceId" : "race_id", "driverId" : "driver_id"})\
    .withColumn('ingestion_date', current_timestamp())

In [0]:
pitstop_final_df.write.format('parquet').mode('overwrite').option('path', f"{silver_folder_path}/pit_stops").save()

In [0]:
spark.read.format('parquet').option('header', True).option('inferSchema', True).load(f"{silver_folder_path}/pit_stops").display()